## IPFS Client
This notebook interacts with our local IPFS node (and, consequently, the configured IPFS private network(s)), using our Python module, as to simulate a Jupyter Server extension (i.e. the backend).

### Python module setup

In [1]:
import sys
!{sys.executable} -m pip install *.whl

Processing ./jcipfsclient-1.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.8 MB/s eta 0:00:0000:01


In [4]:
import jcipfsclient as ipfs

### Verify local communication to our IPFS node

In [2]:
nodeApiUrl = 'http://ipfs.jupyter.localhost:5001'
ipfs.getId(nodeApiUrl)

'12D3KooWGbuQu8v5e3aqehVigAbv6VLLFzYnweXGAXVuGoXsw3RX'

### Join the IPFS network
Using the configuration [obtained via Fabric](./Fabric.ipynb#Configure-IPFS), we will join our local IPFS node to the private IPFS network.

In [3]:
ipfs.joinNetwork(
  nodeApiUrl,
  '../ipfs',
  '/key/swarm/psk/1.0.0/\n/base16/\n63e8d44cb8d738ece5681d42dc918ff882cbe28458d81f64e764f95e3f77929f',
  [
      '/dns4/peer0.pnet0.orga.ipfs.localhost/tcp/4001/p2p/12D3KooWEq2SRZg8zYMjm3vp8G1xettJpZvjF62n58iJu7jhKdzm',
      '/dns4/peer0.pnet0.orgb.ipfs.localhost/tcp/4001/p2p/12D3KooWLy5UaZMMR8rtvxHFa3L9LhvjWwdMBtP9e7BpHjvooiPD',
      '/dns4/peer0.pnet0.orgc.ipfs.localhost/tcp/4001/p2p/12D3KooWByqyhou8Mj5YXTAojVE9TMLqkh6cKqDaEGX4HN5ML3PW'
  ]
)

Verify if our IPFS node can detect any peers on the network:

In [4]:
ipfs.getPeers(nodeApiUrl)

[{'peer': '12D3KooWAgXkQ9G5bka1izzFwfX5tBEZoADFxvSvgfEcSPLn3HiN',
  'addr': '/ip4/172.20.3.2/tcp/4001'},
 {'peer': '12D3KooWEq2SRZg8zYMjm3vp8G1xettJpZvjF62n58iJu7jhKdzm',
  'addr': '/ip4/172.20.3.4/tcp/4001'},
 {'peer': '12D3KooWGbuQu8v5e3aqehVigAbv6VLLFzYnweXGAXVuGoXsw3RX',
  'addr': '/ip4/172.20.3.46/tcp/4001'},
 {'peer': '12D3KooWByqyhou8Mj5YXTAojVE9TMLqkh6cKqDaEGX4HN5ML3PW',
  'addr': '/ip4/172.20.3.5/tcp/4001'},
 {'peer': '12D3KooWLy5UaZMMR8rtvxHFa3L9LhvjWwdMBtP9e7BpHjvooiPD',
  'addr': '/ip4/172.20.3.7/tcp/4001'},
 {'peer': '12D3KooWBXb1BkpMr6hFPuqqp3h19hzf4iZEWGh97Wwtw4hnCXzV',
  'addr': '/ip4/172.20.3.8/tcp/4001'},
 {'peer': '12D3KooWQesTVP2YaGvj5VYDE3fZiW8E1aTDg9TKj4VmwoY8rJzR',
  'addr': '/ip4/172.20.3.9/tcp/4001'}]

### Add a file
Now that we have configured our IPFS network access, we can add a file from our local system (i.e. the JupyterLab backend storage) to IPFS. Once we have added a file to IPFS, we can [publish](./Fabric.ipynb#Add-a-data-description) its metadata on Fabric (note that the file can be automatically encrypted using a random self-generated generated key).

In [9]:
!echo 'This is a test file.' > testorga.txt

In [6]:
ipfs.addFile(nodeApiUrl, './testorga.txt')

{'cid': 'Qmafqh1NmTEto87EvwWe9z5tEpaFjSzgnnXyf65UnffoRG',
 'base64Key': 'z9CYO5kT6X/Jmx6BbS7yq7y2cACDDzCRIc7sgQ7rGZY=',
 'chunkSize': 10485760,
 'cipherMode': 'ChaCha20'}

### Retrieve a file
We can now switch to another JupyterLab instance and, after [repeating](./Fabric.ipynb) the steps to create another Fabric identity and configuring our local IPFS node, [retrieve](./Fabric.ipynb#Retrieve-a-data-description) the file's metadata (CID and encryption info) from Fabric. Then we can use IPFS to download our file.

In [ ]:
filename = 'testorga.txt'
cid = ''
cryptKey = ''
chunkSize = 10485760
cipher = 'ChaCha20'
ipfs.getFile(nodeApiUrl, cid, filename, cryptKey, chunkSize, cipher)

In [ ]:
!cat testorga.txt

### Pinning Service
In the example above our added file is *only* available if our IPFS node is connected to the specified IPFS private network. However, we can use our organization's [provided](./Fabric.ipynb#Configure-IPFS) IPFS Pinning Service to store our (encrypted) file on our organization's IPFS cluster. This guarantees there's always an IPFS peer (or peers) available to provide our file.

In [14]:
cid = 'QmSX7hYKSSGoCn4QLyahHwtyuxW7dHtSQ93gmyHJzGjzBv'
pinningServiceUrl = 'https://cluster0.pnet0.orga.ipfs.localhost:9097'
pinningServiceUser = 'orga'
pinningServicePass = '325298731aB2022aFF0964813762fC'
pinningServiceCert = '../crypto-config/ipfs/orga/pnet0/cluster0/pinsvcapi.crt'
requestId = ipfs.addRemotePin(pinningServiceUrl, cid, pinningServiceUser, pinningServicePass, pinningServiceCert)
print(requestId)

QmSX7hYKSSGoCn4QLyahHwtyuxW7dHtSQ93gmyHJzGjzBv


In [15]:
ipfs.getRemotePinStatus(pinningServiceUrl, requestId, pinningServiceUser, pinningServicePass, pinningServiceCert)

'pinned'

### Storage maintenance
Our local IPFS node contains a copy of all encrypted files we have downloaded. We can manually clean up these files immediately to claim back storage space.

In [ ]:
ipfs.collectGarbage(nodeApiUrl)

**Removing local pins**  
By default, all our added (encrypted) files are always preserved by our local IPFS node (even after a garbage collection). We can manually remove these pins from our node. However, as long as the file's metadata (i.e. the encryption key) [is still available on Fabric](./Fabric.ipynb#Delete-a-data-description), and some other IPFS peer has a copy of the file, the file can still be accessed.

In [ ]:
ipfs.getPins(nodeApiUrl)

In [ ]:
ipfs.rmPin(nodeApiUrl, cid)